In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
import diffxpy.api as de
import statsmodels.stats.multitest as multi
import pickle

In [2]:
adata = sc.read_h5ad(open("/nfs/team283/aa16/KR_NAS/data/CR_cells_AnnData.h5ad", "rb"))

In [3]:
cr_cell_nuclei = np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells'])
cr_cell_reln = np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X)

# Number of nuclei per AOI type:
print('CR cell ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']))
print('Residual ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']))
# RELN expression per AOI type:
print('CR cell ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X))
print('Residual ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'Residual',np.array(adata.var == 'RELN')].X))
# NR2F1 expression in Frontal vs Occipital:
print('Frontal Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X))
print('Occipital Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X))

if np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']) != cr_cell_nuclei:
    raise ValueError()
elif np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X) != cr_cell_reln:
     raise ValueError()
elif np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X) > np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X):
    raise ValueError()
else:
    print('Input data looks sensible.')

CR cell ROIs mean nuclei: 23.445945945945947
Residual ROIs mean nuclei: 136.55714285714285
CR cell ROIs mean RELN expression: 415.3784
Residual ROIs mean RELN expression: 13.985714
Frontal Cortex ROIs mean NR2F1 expression: 19.028572
Occipital Cortex ROIs mean NR2F1 expression: 47.048782
Input data looks sensible.


In [4]:
adata.var_names = adata.var['SYMBOL']

In [5]:
adata = adata[[adata.obs['Pool'].iloc[i] in ('CRcells', 'Residual') for i in range(len(adata.obs['Pool']))],:]

In [6]:
if np.any(np.array([adata.obs['Pool'].iloc[i] not in ('CRcells', 'Residual') for i in range(len(adata.obs['Pool']))])):
    raise ValueError()
elif np.sum(adata.obs['Pool'] == 'CRcells') == len(adata.obs['Pool']):
     raise ValueError()
else:
    print('Input data still makes sense.')
    
adata.obs['Donor'] = [x.split('-')[0] for x in adata.obs['Sample slide ID']]
(adata.obs['Tissue'].astype(str) + ' ' + adata.obs['Pool'].astype(str)
 + ' ' + adata.obs['Donor'].astype(str)).value_counts()

Trying to set attribute `.obs` of view, copying.


Input data still makes sense.


Foetal brain (occipital lobe) Residual Hob7    12
Foetal brain (frontal lobe) Residual Hob8      12
Foetal brain (occipital lobe) CRcells Hob7     12
Foetal brain (frontal lobe) CRcells Hob8       12
Foetal brain (temporal lobe) CRcells Hob7      10
Foetal brain (temporal lobe) CRcells Hob8      10
Foetal brain (temporal lobe) Residual Hob7     10
Foetal brain (parietal lobe) Residual Hob8      9
Foetal brain (temporal lobe) Residual Hob8      9
Foetal brain (parietal lobe) CRcells Hob8       9
Foetal brain (occipital lobe) Residual Hob8     8
Foetal brain (occipital lobe) CRcells Hob8      8
Foetal brain (parietal lobe) CRcells Hob7       7
Foetal brain (frontal lobe) CRcells Hob7        6
Foetal brain (frontal lobe) Residual Hob7       5
Foetal brain (parietal lobe) Residual Hob7      5
dtype: int64

In [7]:
adata.X = adata.layers['X_corrected_naive_cpm']
res = de.test.rank_test(
    data=adata,
    grouping="Pool",
    is_logged=False
).summary()            
    
res['log2_mean_CRcells'] = np.array(np.log2(np.mean(adata[adata.obs['Pool'] == 'CRcells',].X, axis = 0)))
res['log2_mean_Residuals'] = np.array(np.log2(np.mean(adata[adata.obs['Pool'] == 'Residual',].X, axis = 0)))
res['log2_fold_change'] =  res['log2_mean_CRcells'] - res['log2_mean_Residuals']

subset1 = [adata.obs['Pool'].iloc[j] == 'CRcells' and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
subset2 = [adata.obs['Pool'].iloc[j] == 'Residual' and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
res['log2_mean_Hob7_CRcells'] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
res['log2_mean_Hob7_Residuals'] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
res['log2_fold_change_Hob7'] =  res['log2_mean_Hob7_CRcells'] - res['log2_mean_Hob7_Residuals']

subset1 = [adata.obs['Pool'].iloc[j] == 'CRcells' and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
subset2 = [adata.obs['Pool'].iloc[j] == 'Residual' and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
res['log2_mean_Hob8_CRcells'] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
res['log2_mean_Hob8_Residuals'] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
res['log2_fold_change_Hob8'] =  res['log2_mean_Hob8_CRcells'] - res['log2_mean_Hob8_Residuals']

res = res.sort_values('pval')
    
res_naive = res

/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log2


In [8]:
pickle.dump(res_naive, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_CRvsBackground_DE_naive.p", "wb" ))

In [9]:
# NR2F1 expression in Frontal vs Occipital:
print('Frontal Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X))
print('Occipital Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X))

if np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']) > np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']):
    raise ValueError()
elif np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X) < np.mean(adata[adata.obs['Pool'] == 'Residual',np.array(adata.var == 'RELN')].X):
    raise ValueError()
elif np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X) > np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X):
    raise ValueError()
else:
    print('Input data still looks sensible.')

Frontal Cortex ROIs mean NR2F1 expression: 168.5970397879765
Occipital Cortex ROIs mean NR2F1 expression: 399.1507441825179
Input data still looks sensible.


In [10]:
adata.X = adata.layers['X_corrected_cc_cpm']
res = de.test.rank_test(
    data=adata,
    grouping="Pool",
    is_logged=False
).summary()            
    
res['log2_mean_CRcells'] = np.array(np.log2(np.mean(adata[adata.obs['Pool'] == 'CRcells',].X, axis = 0)))
res['log2_mean_Residuals'] = np.array(np.log2(np.mean(adata[adata.obs['Pool'] == 'Residual',].X, axis = 0)))
res['log2_fold_change'] =  res['log2_mean_CRcells'] - res['log2_mean_Residuals']

subset1 = [adata.obs['Pool'].iloc[j] == 'CRcells' and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
subset2 = [adata.obs['Pool'].iloc[j] == 'Residual' and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
res['log2_mean_Hob7_CRcells'] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
res['log2_mean_Hob7_Residuals'] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
res['log2_fold_change_Hob7'] =  res['log2_mean_Hob7_CRcells'] - res['log2_mean_Hob7_Residuals']

subset1 = [adata.obs['Pool'].iloc[j] == 'CRcells' and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
subset2 = [adata.obs['Pool'].iloc[j] == 'Residual' and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
res['log2_mean_Hob8_CRcells'] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
res['log2_mean_Hob8_Residuals'] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
res['log2_fold_change_Hob8'] =  res['log2_mean_Hob8_CRcells'] - res['log2_mean_Hob8_Residuals']

res = res.sort_values('pval')
    
res_cc = res

/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log2


In [11]:
pickle.dump(res_cc, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_CRvsBackground_DE_cc.p", "wb" ))

Save all data on pairwise differential expression in CR cells in one data frame:

In [13]:
res_cc

,gene,pval,qval,log2fc,mean,zero_mean,zero_variance,log2_mean_CRcells,log2_mean_Residuals,log2_fold_change,log2_mean_Hob7_CRcells,log2_mean_Hob7_Residuals,log2_fold_change_Hob7,log2_mean_Hob8_CRcells,log2_mean_Hob8_Residuals,log2_fold_change_Hob8
2251,CBLN1,1.450452e-25,1.148978e-21,-4.706766,266.255303,False,False,8.965803,4.259037,4.706766,9.247315,4.561271,4.686044,8.655784,3.943847,4.711937
146,ACHE,2.786022e-25,1.148978e-21,-4.027922,238.147090,False,False,8.774857,4.746935,4.027922,9.022046,4.898604,4.123442,8.509991,4.605551,3.904439
10095,NDNF,2.793576e-25,1.148978e-21,-5.319864,879.968265,False,False,10.708011,5.388147,5.319864,10.932392,5.950913,4.981480,10.471818,4.646928,5.824891
13225,RELN,4.112891e-25,1.148978e-21,-6.234965,4561.039225,False,False,13.097613,6.862648,6.234965,13.352103,7.024564,6.327539,12.823347,6.710614,6.112733
10770,OLFM1,4.488515e-25,1.148978e-21,-3.444729,1080.395968,False,False,10.917629,7.472900,3.444729,11.362636,7.789409,3.573227,10.352163,7.139057,3.213106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17210,UBE2D4,1.000000e+00,1.000000e+00,-0.152974,17.009014,False,False,4.160567,4.007593,0.152974,4.186910,3.572056,0.614854,4.136509,4.293808,-0.157299
3629,CSNK1A1,1.000000e+00,1.000000e+00,-0.121592,62.979918,False,False,6.034649,5.913057,0.121592,6.076901,5.845917,0.230984,5.995646,5.967269,0.028378
16707,TP53BP1,1.000000e+00,1.000000e+00,0.156778,107.887485,False,False,6.675043,6.831821,-0.156778,6.649987,7.200922,-0.550935,6.697165,6.425387,0.271778
9047,MARK2,1.000000e+00,1.000000e+00,-0.089169,50.826970,False,False,5.710181,5.621011,0.089169,5.987643,5.408992,0.578651,5.405620,5.778174,-0.372554


In [14]:
res_cc = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_CRvsBackground_DE_cc.p", "rb" ))
res_naive = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_CRvsBackground_DE_naive.p", "rb" ))
    
res_tidy = pd.DataFrame(index = list(range(len(res_cc['gene']))), columns = ('gene', 'qval_cc', 'qval_naive',
                                   'log2_mean_wta_CRcells', 'log2_mean_wta_Residuals',                                     
                                   'log2fc_cc', 'log2fc_naive',
                                   'log2fc_Hob7_cc', 'log2fc_Hob8_cc',
                                   'log2fc_Hob7_naive', 'log2fc_Hob8_naive'))
    
for i in range(len(res_cc['gene'])):
    res_tidy['gene'].iloc[i]  = res_cc['gene'].iloc[i]

    index_naive = res_naive['gene'] == res_cc['gene'].iloc[i]

    if res_naive['gene'].loc[index_naive].iloc[0] != res_cc['gene'].iloc[i]:
        print(res_naive['gene'].loc[index_naive])
        print(res_cc['gene'].iloc[i])
        raise ValueError()

    res_tidy['qval_cc'].iloc[i] = res_cc['qval'].iloc[i]
    res_tidy['qval_naive'].iloc[i]  = res_naive['qval'].loc[index_naive].iloc[0]

    res_tidy['log2_mean_wta_CRcells'].iloc[i] = res_cc['log2_mean_CRcells'].iloc[i]
    res_tidy['log2_mean_wta_Residuals'].iloc[i] = res_cc['log2_mean_Residuals'].iloc[i]

    res_tidy['log2fc_cc'].iloc[i]  = res_cc['log2_fold_change'].iloc[i]
    res_tidy['log2fc_naive'].iloc[i]  = res_naive['log2_fold_change'].loc[index_naive].iloc[0]

    res_tidy['log2fc_Hob7_cc'].iloc[i]  = res_cc['log2_fold_change_Hob7'].iloc[i]
    res_tidy['log2fc_Hob7_naive'].iloc[i]  = res_naive['log2_fold_change_Hob7'].loc[index_naive].iloc[0]

    res_tidy['log2fc_Hob8_cc'].iloc[i]  = res_cc['log2_fold_change_Hob8'].iloc[i]
    res_tidy['log2fc_Hob8_naive'].iloc[i]  = res_naive['log2_fold_change_Hob8'].loc[index_naive].iloc[0]

    if res_tidy['log2fc_cc'].iloc[i] != (res_tidy['log2_mean_wta_CRcells'].iloc[i] - res_tidy['log2_mean_wta_Residuals'].iloc[i]):
        raise ValueError()

In [17]:
res_tidy

,gene,qval_cc,qval_naive,log2_mean_wta_CRcells,log2_mean_wta_Residuals,log2fc_cc,log2fc_naive,log2fc_Hob7_cc,log2fc_Hob8_cc,log2fc_Hob7_naive,log2fc_Hob8_naive
0,CBLN1,0.0,0.0,8.965803,4.259037,4.706766,4.285876,4.686044,4.711937,3.925495,4.784722
1,ACHE,0.0,0.0,8.774857,4.746935,4.027922,3.427797,4.123442,3.904439,3.304971,3.553356
2,NDNF,0.0,0.0,10.708011,5.388147,5.319864,5.388882,4.98148,5.824891,4.874258,6.172411
3,RELN,0.0,0.0,13.097613,6.862648,6.234965,5.933265,6.327539,6.112733,5.881612,5.97873
4,OLFM1,0.0,0.0,10.917629,7.4729,3.444729,3.125051,3.573227,3.213106,3.123395,3.098799
...,...,...,...,...,...,...,...,...,...,...,...
18672,UBE2D4,1.0,0.444343,4.160567,4.007593,0.152974,0.305367,0.614854,-0.157299,0.527823,0.099586
18673,CSNK1A1,1.0,0.995224,6.034649,5.913057,0.121592,0.048353,0.230984,0.028378,0.111507,-0.008585
18674,TP53BP1,1.0,0.853438,6.675043,6.831821,-0.156778,-0.239636,-0.550935,0.271778,-0.681393,0.256515
18675,MARK2,1.0,0.976762,5.710181,5.621011,0.089169,0.024269,0.578651,-0.372554,0.244202,-0.172866


In [18]:
pickle.dump(res_tidy, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CRvsBackground_DE_allResults.p", "wb" ))

In [19]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('/nfs/team283/aa16/KR_NAS/0_publication_tables/CRvsBackground_DE_results.p.xlsx', engine='xlsxwriter')


res_tidy.to_excel(writer, sheet_name= 'CRcells_vs_Residuals')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
res_sc = pickle.load(open("/nfs/team283/brainData/human_fetal/Bhaduri2021/CR_cells_pairwiseDE.p", "rb" ))
res_cc = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_cc.p", "rb" ))
res_naive = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_naive.p", "rb" ))

In [ ]:
res_naive[list(res_naive.keys())[0]]

In [ ]:
res_cc[list(res_cc.keys())[0]]